These codes are used to explore the use of cv2 with TuringComplete  
game -> https://store.steampowered.com/app/1444480/Turing_Complete/

Game Description
This game is based on a powerful circuit simulator. This circuit simulator allows you to give free rein to your imagination and pass through the levels with different solutions or build your own computer the way you like. You can connect as many components as you like to your computer such as displays, timers, sound components, etc., as well as receive data sent from real-life keyboards and networks. You can even design your own proprietary assembly language for your own computer.

In the following code I will use cv2 and keyboard input to try to get through the maze challenges in the game, I will rewrite the assembly code from the game in python before trying to train the model to pass the game using q larning

run in win10( 2k 125% ) in game "the maze" (150% biggest window)

<img src=".\images\TuringCompleteimage.png" width="500" />

1.Use cv2 to detect the surrounding :)

In [22]:
import time
import pyautogui
import cv2
import numpy as np


def capture_screen(region=None):
    screenshot = pyautogui.screenshot(region=region) # screenshot
    img_bgr = np.array(screenshot) # convert to numpy array
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB) # convert
    return img_rgb


def findAndShow(templatewall,threshold=0.9):
    iswall = [0, 0, 0, 0]#["up","left","down","right"]
    for i in range(4):
        screen = capture_screen()
        time.sleep(0.01)
        # 使用temlatewall匹配确认前面有没有wall
        result = cv2.matchTemplate(screen, templatewall, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        if max_val > threshold:
            iswall[i]= 1
        else:
            iswall[i] = 0
        pyautogui.press('left')
    printIsWall(iswall)
    return iswall


def printIsWall(iswall):
    # 1 is wall 0 is not wall. 
    # Example: When the array is [1,0,1,1] the resultw will be 
    # up is wall, left not wall, down is wall, right is wall
    directions = ["up","left","down","right"]
    for i in range(len(iswall)):
        if iswall[i] == 1:
            print(f"{directions[i]} is wall")
        else:
            print(f"{directions[i]} not wall")


def main():
    templatewall = cv2.imread(".\\images\\tcwall.png",cv2.IMREAD_COLOR)
    if templatewall is None:
        print("Error: image not be loaded.")
        return 0 
    #cv2.namedWindow("Screen", cv2.WINDOW_NORMAL)
    # while True:
    #     findAndShow()
    #     if cv2.waitKey(1) & 0xFF == ord('q'):
    #         break
    print("ready?")
    time.sleep(2)
    print("run!")
    findAndShow(templatewall)

if __name__ == "__main__":
    main()



ready?
run!
up not wall
left not wall
down not wall
right not wall


Complete the game using the left-first principle (write the DIY assembly language code in the game as python)

In [3]:
import time
import pyautogui
import cv2
import numpy as np
import keyboard 

def capture_screen(region=None):
    screenshot = pyautogui.screenshot(region=region) # screenshot
    img_bgr = np.array(screenshot) # convert to numpy array
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB) # convert
    return img_rgb


def findAndShow(templatewall,templatedoor,threshold=0.9):
    iswall = [0, 0, 0, 0]#["front","left","back","right"]
    isDoor = [0, 0, 0, 0]#["front","left","back","right"]
    for i in range(4):
        screen = capture_screen()
        #time.sleep(0.01)
        # 使用temlatewall匹配确认前面有没有wall
        resultwall = cv2.matchTemplate(screen, templatewall, cv2.TM_CCOEFF_NORMED)
        min_val, max_valwall, min_loc, max_loc = cv2.minMaxLoc(resultwall)
        if max_valwall > threshold:
            iswall[i]= 1
        else:
            iswall[i] = 0
        # 使用temlatewall匹配确认前面有没有door
        resultdoor = cv2.matchTemplate(screen, templatedoor, cv2.TM_CCOEFF_NORMED)
        min_val, max_valdoor, min_loc, max_loc = cv2.minMaxLoc(resultdoor)
        if max_valdoor > threshold:
            isDoor[i]= 1
        else:
            isDoor[i] = 0
        pyautogui.press('left')
    return iswall , isDoor


def main():
    templatewall = cv2.imread(".\\images\\tcwall.png",cv2.IMREAD_COLOR)
    templatedoor = cv2.imread(".\\images\\tcdoor.png",cv2.IMREAD_COLOR)
    if templatewall is None or templatedoor is None:
        print("Error: images not be loaded.")
        return 0 
    print("ready?")
    time.sleep(2)
    print("run!")
    while True:
        if keyboard.is_pressed('q'):
            print("Program stop!")
            break

        result_wall, result_door = findAndShow(templatewall,templatedoor)
        for i in range(4):
            if result_door[i] == 1 :
                print("you find door ! :)")
                return 1 
        #move
        if result_wall[1] == 0 :
            print("go left")
            pyautogui.press('left')
            pyautogui.press('up')
        elif result_wall[0] == 0 :
            print("go front")
            pyautogui.press('up')
        elif result_wall[3] == 0 :
            print("go right")
            pyautogui.press('right')
            pyautogui.press('up')
        elif result_wall[2] == 0 :
            print("go back")
            pyautogui.press('right')
            pyautogui.press('right')
            pyautogui.press('up')
        else :
            print("Error :(")
            return 0 
        


if __name__ == "__main__":
    main()


ready?
run!
go front
go right
go front
go left
go front
go left
go front
you find door ! :)


We can try to use q-learning in this maze game now

Each position and direction in the maze can be considered as a state, and each movement as an action. 